In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese_qingyun/main')

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.2
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx


def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_seq2seq_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_export/1587451865/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    print('Rank_{}:'.format(j+1), ' '.join(indices))
  print('%.2f sec' % (time.time() - t0))
  print()

Input:你好
Rank_1: 你 好
Rank_2: 大 美 女
Rank_3: 我 很 好 ， 你 呢 ， 你 怎 么 样
0.57 sec

Input:早上好
Rank_1: 早 上 好 啊 ， 苦 逼 的 一 天 开 始 了
Rank_2: 早 安
Rank_3: 你 好 呀
0.10 sec

Input:晚上好
Rank_1: 你 好 你 好 ~
Rank_2: 你 好 你 好
Rank_3: 你 好
0.09 sec

Input:再见
Rank_1: 嗯 ， 再 见
Rank_2: 拜 拜 ~
Rank_3: 8 8 ， 要 想 我 哦
0.07 sec

Input:好久不见
Rank_1: 是 啊 ， 蛮 久 的 了 ， 好 想 你 啊
Rank_2: 是 啊 ， 蛮 久 的 了 ， 好 想 你
Rank_3: 是 啊 ， 蛮 久 的 了
0.09 sec

Input:想死你了
Rank_1: 我 也 想 你 ， 你 知 道 的 ， 我 永 远 不 会 忘 了 你
Rank_2: 我 也 想 你 啦
Rank_3: 我 也 是
0.09 sec

Input:谢谢你
Rank_1: 不 用 谢 啦
Rank_2: 那 么 客 气 干 嘛
Rank_3: 人 又 不 斯 文 ， 装 得 这 么 客 气 干 嘛 呢
0.12 sec

Input:爱你
Rank_1: 哦
Rank_2: 有 多 爱 呢
Rank_3: 你 打 算 向 我 求 婚 吗
0.12 sec

Input:你叫什么名字
Rank_1: 我 不 叫
Rank_2: 我 好
Rank_3: 不 知 道
0.06 sec

Input:你几岁了
Rank_1: 没 年 龄
Rank_2: 女 生 的 年 龄 是 可 以 随 便 问 的 吗
Rank_3: 你 是 不 是 觉 得 这 日 子 过 得 太 无 聊 了 哦
0.08 sec

Input:现在几点了
Rank_1: 现 在 时 刻 1 6 点 5 3 分
Rank_2: 现 在 时 刻 1 7 点 5 7 分
Rank_3: 现 在 时 刻 1 6 点 5 7 分
0.06 sec

Input:今天天气怎么样
Rank_1: 好 好
Rank_2: 长 得 还 行
Rank_3: 好 好 味
0.13 sec
